# Notebook Summary
Creating a dataframe for songs that contain ISRC codes

In [20]:
import autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pickle
import requests
import base64
import datetime
import numpy as np
import pandas

# Load Songs

In [22]:
# Load Pickled Dataframe from Cleaning Notebook into a DataFrame
path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\Pickle.pkl"

df = pickle.load(open(path,'rb'))
df.head()

,Playlist Add Date,Record Label,UPC,Album Name,ISRC,name,Peak Position,Peak Date,Rank,Historical Positions,Time on Chart,Artist(s),tiktok_track_id,Velocity,Historical Posts,TikTok Link,Release Date,Position Change
0,2020-08-23T00:00:00.000Z,JVKE,193436230483,Upside Down,QM24S2008283,All TikTok Mashup (JVKE - Upside Down),1,2020-09-02,1,"24, 1",56,JVKE 🌩,All-TikTok-Mashup-JVKE-Upside-Down-68463501735...,,"100000, 300000",https://www.tiktok.com/music/All-TikTok-Mashup...,2020-08-17,23.0
0,2020-11-03T00:00:00.000Z,ColliPark Music,859705372161,Alley: The Return of the Ying Yang Twins,USW9J1000104,Say I Yi Yi,1,2020-11-02,1,"5, 2, 2, 2, 3, 1",23,D-Roc,Say-I-Yi-Yi-6883981933215369989,,"105900, 148200, 177600, 229200, 200000, 200000",https://www.tiktok.com/music/Say-I-Yi-Yi-68839...,2002-03-25,2.0
0,2020-11-01T00:00:00.000Z,MFLP,"829410801386, 888072351684, 600406849243, 8880...","Halloween Howls, Halloween Howls: Fun & Scary ...",USB440453203,"Spooky, Scary Skeletons",1,2020-10-31,1,"15, 8, 20, 8, 18, 13, 23, 11, 22, 20, 29, 9, 3...",34,Andrew Gold,Spooky-Scary-Skeletons-6602666381370460933,2.14286,"100000, 100000, 100000, 100000, 100000, 100000...",https://www.tiktok.com/music/Spooky-Scary-Skel...,2006-10-02,5.0
0,2020-10-29T00:00:00.000Z,"Astroknot Sounds, Opposition",193436238021,Garage Room Freestyle,QM24S2009905,Garage Room Freestyle (Reel It In),1,2020-10-28,1,"23, 15, 5, 4, 19, 17, 14, 22, 19, 3, 10, 6, 15, 1",32,Number9ok,Garage-Room-Freestyle-Reel-It-In-6876123370077...,1.85714,"40000, 81500, 123900, 116700, 74700, 72700, 78...",https://www.tiktok.com/music/Garage-Room-Frees...,2020-10-13,14.0
0,2020-10-26T00:00:00.000Z,Not specified,886448446026,"Ho, why is you here ?",USRC12002532,May I,1,2020-10-25,1,"33, 24, 35, 32, 23, 41, 34, 30, 29, 28, 28, 37...",38,Flomilli,May-I-6852015601389438978,9.42857,"38200, 34100, 39700, 31200, 35900, 44400, 4080...",https://www.tiktok.com/music/May-I-68520156013...,2020-07-23,76.0


### Let's drop all rows where there is no ISRC.  This is a identifier to each song, that we will send with each Spotify request

In [23]:
# We start with 15,376 songs
df.shape

(15376, 18)

In [24]:
df['ISRC']=df['ISRC'].str.strip().replace('',np.nan)

In [25]:
df.dtypes

Playlist Add Date        object
Record Label             object
UPC                      object
Album Name               object
ISRC                     object
name                     object
Peak Position             int64
Peak Date                object
Rank                      int64
Historical Positions     object
Time on Chart             int64
Artist(s)                object
tiktok_track_id          object
Velocity                 object
Historical Posts         object
TikTok Link              object
Release Date             object
Position Change         float64
dtype: object

In [27]:
df.head(10)

,Playlist Add Date,Record Label,UPC,Album Name,ISRC,name,Peak Position,Peak Date,Rank,Historical Positions,Time on Chart,Artist(s),tiktok_track_id,Velocity,Historical Posts,TikTok Link,Release Date,Position Change
0,2020-08-23T00:00:00.000Z,JVKE,193436230483,Upside Down,QM24S2008283,All TikTok Mashup (JVKE - Upside Down),1,2020-09-02,1,"24, 1",56,JVKE 🌩,All-TikTok-Mashup-JVKE-Upside-Down-68463501735...,,"100000, 300000",https://www.tiktok.com/music/All-TikTok-Mashup...,2020-08-17,23.0
0,2020-11-03T00:00:00.000Z,ColliPark Music,859705372161,Alley: The Return of the Ying Yang Twins,USW9J1000104,Say I Yi Yi,1,2020-11-02,1,"5, 2, 2, 2, 3, 1",23,D-Roc,Say-I-Yi-Yi-6883981933215369989,,"105900, 148200, 177600, 229200, 200000, 200000",https://www.tiktok.com/music/Say-I-Yi-Yi-68839...,2002-03-25,2.0
0,2020-11-01T00:00:00.000Z,MFLP,"829410801386, 888072351684, 600406849243, 8880...","Halloween Howls, Halloween Howls: Fun & Scary ...",USB440453203,"Spooky, Scary Skeletons",1,2020-10-31,1,"15, 8, 20, 8, 18, 13, 23, 11, 22, 20, 29, 9, 3...",34,Andrew Gold,Spooky-Scary-Skeletons-6602666381370460933,2.14286,"100000, 100000, 100000, 100000, 100000, 100000...",https://www.tiktok.com/music/Spooky-Scary-Skel...,2006-10-02,5.0
0,2020-10-29T00:00:00.000Z,"Astroknot Sounds, Opposition",193436238021,Garage Room Freestyle,QM24S2009905,Garage Room Freestyle (Reel It In),1,2020-10-28,1,"23, 15, 5, 4, 19, 17, 14, 22, 19, 3, 10, 6, 15, 1",32,Number9ok,Garage-Room-Freestyle-Reel-It-In-6876123370077...,1.85714,"40000, 81500, 123900, 116700, 74700, 72700, 78...",https://www.tiktok.com/music/Garage-Room-Frees...,2020-10-13,14.0
0,2020-10-26T00:00:00.000Z,Not specified,886448446026,"Ho, why is you here ?",USRC12002532,May I,1,2020-10-25,1,"33, 24, 35, 32, 23, 41, 34, 30, 29, 28, 28, 37...",38,Flomilli,May-I-6852015601389438978,9.42857,"38200, 34100, 39700, 31200, 35900, 44400, 4080...",https://www.tiktok.com/music/May-I-68520156013...,2020-07-23,76.0
0,2020-12-24T00:00:00.000Z,NaN,NaN,NaN,NaN,Trên Tình Bạn Dưới Tình Yêu,1,2020-12-23,1,"109, 126, 204, 281, 1",80,MIN,Trên-Tình-Bạn-Dưới-Tình-Yêu-6892203921...,,"4900, 4100, 2200, 2200, 450600",https://www.tiktok.com/music/Trên-Tình-Bạn-...,Invalid date,280.0
0,2020-12-23T00:00:00.000Z,NaN,NaN,NaN,NaN,Set 2.0: Homenagem aos Relíquias,1,2020-12-22,1,"2923, 34, 43, 48, 56, 53, 76, 79, 167, 96, 102...",43,DJ Matt D & MC Leozinho ZS & MC Vinny & Menor ...,Set-20-Homenagem-aos-Relíquias-688696135642828...,10.7143,"100, 14800, 11800, 11500, 10100, 7900, 5700, 5...",https://www.tiktok.com/music/Set-20-Homenagem-...,Invalid date,135.0
0,2020-11-04T00:00:00.000Z,"GT Digital, Thizz Latin Denver, Kreepa",194690079610,Cold As Ice,USUYG1287925,Oh No,1,2020-11-06,1,"140, 51, 40, 27, 24, 4, 1",112,Kreepa,Oh-No-6586947002464996102,,"3900, 9900, 18800, 30100, 59600, 101400, 129400",https://www.tiktok.com/music/Oh-No-65869470024...,2019-12-12,3.0
0,2020-12-18T00:00:00.000Z,NaN,NaN,NaN,NaN,Em Băng Qua (Remix),1,2020-12-17,1,"3245, 3356, 3312, 1742, 3222, 3408, 4371, 3569...",75,Lập Nguyên,Em-Băng-Qua-Remix-6894582266408798978,479.286,"55, 47, 51, 101, 82, 60, 17, 34, 24, 45, 39, 3...",https://www.tiktok.com/music/Em-Băng-Qua-Remix...,Invalid date,84.0
0,2020-06-23T00:00:00.000Z,Albrew,"859737184626, 195833186554","Suling Sakti Spongebob, Suling Sakti Spongebob",TCAEQ2062800,Suling Sakti Spongebob (New),1,2020-06-22,1,1,4,🄰🄻🄱🅁🄴🅆 🦊,Suling-Sakti-Spongebob-New-6779411527796607746,,400000,https://www.tiktok.com/music/Suling-Sakti-Spon...,2020-02-06,-1.0


In [28]:
df = df.dropna(subset=['ISRC'])

In [29]:
# About 5,5000 songs were dropped

df.shape

(9930, 18)

In [32]:
# Pickle the cleaned dataframe

# Designate path

path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\Spotify_Pickle.pkl"
with open(path, 'wb') as f:
    pickle.dump(df, f)